#### Load packages
Let's load the required packages. 

In [1]:
import os

import cv2
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from sklearn.utils import shuffle

print('tf version:', tf.__version__)

tf.keras.backend.set_image_data_format('channels_first')
np.random.seed(1234)
tf.random.set_seed(1234)
np.set_printoptions(threshold=5)

%matplotlib inline
%load_ext autoreload
%autoreload 2


tf version: 2.0.0-rc1


In [2]:
def build_mlp_model(input_size, output_size):
    '''
    Builds the layers for LSTM model.

    args:
        - input_size: (tuple) (sequence len, num keypoints)
        - output_size: (int) number of classes
    '''
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(input_size, activation='relu'),
        tf.keras.layers.Dense(output_size)
    ])
    return model


In [3]:
@tf.function
def compute_loss(label, pred):
    return criterion(label, pred)

@tf.function
def train_step(x, t):
    with tf.GradientTape() as tape:
        preds = model(x)
        loss = compute_loss(t, preds)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss)
    train_acc(t, preds)

    return preds

@tf.function
def test_step(x, t):
    preds = model(x)
    loss = compute_loss(t, preds)
    test_loss(loss)
    test_acc(t, preds)

    return preds

In [4]:
'''
Load data
'''
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train.reshape(-1, 784) / 255).astype(np.float32)
x_test = (x_test.reshape(-1, 784) / 255).astype(np.float32)
y_train = np.eye(10)[y_train].astype(np.float32)
y_test = np.eye(10)[y_test].astype(np.float32)

'''
Build model
'''
model = build_mlp_model(200, 10)
criterion = tf.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

'''
Train model
'''
epochs = 10
batch_size = 100
n_batches = x_train.shape[0] // batch_size

train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.CategoricalAccuracy()
test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.CategoricalAccuracy()

for epoch in range(epochs):

    _x_train, _y_train = shuffle(x_train, y_train, random_state=42)

    for batch in range(n_batches):
        start = batch * batch_size
        end = start + batch_size
        train_step(_x_train[start:end], _y_train[start:end])

    if epoch % 5 == 4 or epoch == epochs - 1:
        preds = test_step(x_test, y_test)
        print('Epoch: {}, Valid Cost: {:.3f}, Valid Acc: {:.3f}'.format(
            epoch+1,
            test_loss.result(),
            test_acc.result()
        ))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch: 5, Valid Cost: 10.588, Valid Acc: 0.218
Epoch: 10, Valid Cost: nan, Valid Acc: 0.158
